<a href="https://colab.research.google.com/github/sireesha-vakada/scisumm-corpus/blob/master/nla_project_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import csv
import xml.etree.ElementTree as ET
import nltk
nltk.download('punkt')
from nltk.util import ngrams
import re 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**PRE**-**PROCESSING**

In [0]:
def parseXML(xmlfile):
  tree = ET.parse(xmlfile)
  root = tree.getroot()
  heads = []
  # print ("hi")
  # print (root.tag)
  ref_sent = []
  for child in root:
    # print (child.tag, child.attrib)
    for each in child:
      ref_sent.append(each.text)
  
  return ref_sent
    

In [0]:
def extract_string(s):
  s = s.split(">")
  t = []
  # print ("s = ", s)
  i = 0
  for each in s:
    if(i%2 != 0):
      temp = str(each)
      temp = temp.split("<")[0]
      # print (temp)
      t.append(temp)
    i = i+1
  
  return t
      

In [0]:
def get_citations(file):
  file_read = open(file, "r")
  citations = file_read.readlines()
  total_cit = []
  for each in citations:
    if each != "\n":
      # print (each)
      tokens = each.split("|")
      ref_article = each.partition("Reference Article:")[2]
      ref_article = ref_article.partition("|")[0]

      cit_article = each.partition("Citing Article:")[2]
      cit_article = cit_article.partition("|")[0]
      
      cit_text = each.partition("Citation Text:")[2]
      cit_text = cit_text.partition("|")[0]

      ref_text = each.partition("Reference Text:")[2]
      ref_text = ref_text.partition("|")[0]

      ref_texts = extract_string(ref_text)
      cit_texts = extract_string(cit_text) 

      # print("cit_text = ", len(cit_texts))
      string = ""
      if(len(cit_texts) > 1):
        for i in range(len(cit_texts)):
          if i > 0:
            string = string[:-1] + " and " + cit_texts[i]
          else:
            string = cit_texts[i]
        print (string)
      else:
        string = str(cit_texts[0])
      
      total_cit.append(string)
    
  return total_cit
      
      


**DATA EXTRACTION**

In [66]:

ref_sentences = parseXML("C00-2123.xml")
cit_sentences = get_citations("C00-2123.txt")

print (len(ref_sentences))
print (len(cit_sentences))

Under this constraint, many researchers had contributed algorithms and associated pruning strategies, such as Berger et al and (1996), Och et al and (2001), Wang and Waibel (1997), Tillmann and Ney (2000) GarciaVarea and Casacuberta (2001) and Germann et al.
This article will present a DP-based beam search decoder for the IBM4 translation model and A preliminary version of the work presented here was published in Tillmann and Ney (2000).
It is faster because the search problem for noisy- channel models is NP-complete (Knight, 1999), and even the fastest dynamic-programming heuristics used in statistical MT (Niessen et al., 1998; Till- mann and Ney, 2000), are polynomial in J —for in p(v1, w and , wm−1, um
203
18


**TF_IDF VECTORS**

In [67]:
cit = cit_sentences[0]
ref = ref_sentences[0]
len_ref = len(ref_sentences)
len_cit = len(cit_sentences)
all_sent = []

for each in ref_sentences:
  all_sent.append(each)

for each in cit_sentences:
  all_sent.append(each)

n_gram = 1;
vectorizer = TfidfVectorizer(ngram_range = (n_gram,n_gram))
tf_df = vectorizer.fit_transform(all_sent) 
X = tf_df.toarray()
print (len(vectorizer.get_feature_names()))
print (X.shape)


997
(221, 997)


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial

def cosine(a,b):
  ans = 1 - spatial.distance.cosine(a,b)
  return ans


In [69]:
#cosine similarity
import numpy


def find_sent(features, k):
  
  for i in range(len_cit):
    maxi = 0
    index = 0
    value = []
    indices = []
    for j in range(len_ref):
      a = features[j]
      b = features[i + len_ref]
      # print(i+len_ref, j)
      val = cosine(a,b)
      if(val  > 0):
        value.append(val)
        indices.append(j)

      if(val > maxi):
        maxi = val
        index = j
    # print ("max_val = ", maxi)
    # print ("cit_sent = ", cit_sentences[i])
    # print ("ref_sent = ", ref_sentences[index])
    s = numpy.array(value)
    t = numpy.argsort(s)

    if(i == 2):
      # print (value)
      # print (t)
      # print (indices)
      print ("cit_sent = ", cit_sentences[i])
      if (k > len(t)):
        k1 = len(t)
      else:
        k1 = k

      for each in range(k1):
        ind = indices[t[-(each+1)]]
        # print (ind)
        print (value[t[-(each+1)]])
        print ("ref_text = ", ref_sentences[ind])
        

    


# tf-idf results
find_sent(X, 5)


cit_sent =  The decoding methods presented in this paper explore the partial candidate translation hypotheses greedily, as presented in Tillmann and Ney (2000) and Och et al.
0.30608072060668
ref_text =  The details are given in (Och and Ney, 2000).
0.27622423694666665
ref_text =  We show translation results for three approaches: the monotone search (MonS), where no word reordering is allowed (Tillmann, 1997), the quasimonotone search (QmS) as presented in this paper and the IBM style (IbmS) search as described in Section 3.2.
0.25758932627232634
ref_text =  We use a solution to this problem similar to the one presented in (Och et al., 1999), where target words are joined during training.
0.25071769987692694
ref_text =  This approach is compared to another reordering scheme presented in (Berger et al., 1996).
0.24962068222017642
ref_text =  In this paper, we have presented a new, eÆcient DP-based search procedure for statistical machine translation.


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [70]:
#word2vec model

import gensim 
from gensim.models import Word2Vec 

word2vec_voc = []
for i in ref_sentences:
  temp = []
  for j in word_tokenize(i):
    temp.append(j.lower())
  word2vec_voc.append(temp)

for i in cit_sentences:
  temp = []
  for j in word_tokenize(i):
    temp.append(j.lower())
  word2vec_voc.append(temp)


CBOW_model = gensim.models.Word2Vec(word2vec_voc, min_count =1,  size=100,  window=5)

print(CBOW_model.similarity('researchers', 'language'))
print(CBOW_model.corpus_total_words)


0.44042498
6070


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [71]:
print(len(word2vec_voc))
# print(CBOW_model.accuracy)
data = CBOW_model["language"]
print (data.shape)

221
(100,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [72]:
# all_sent - consists of all the sentences
import numpy as np

total_voc = []

for sent in all_sent:
  for word in word_tokenize(sent):
    total_voc.append(word.lower())

print (len(total_voc))
tf_awv_feature = []

#only based on tf and with wor2vec weighted average similarities

for sent in all_sent:
  temp = np.zeros(100)
  for word in word_tokenize(sent):
    word = word.lower()
    temp = temp + CBOW_model[word] 
  temp = temp/len(word_tokenize(sent))
  tf_awv_feature.append(temp)

print(len(tf_awv_feature))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


6070
221


In [77]:
#another cosine similarity with word2vec

find_sent(tf_awv_feature,  15)

cit_sent =  The decoding methods presented in this paper explore the partial candidate translation hypotheses greedily, as presented in Tillmann and Ney (2000) and Och et al.
0.9996862113736001
ref_text =  To explicitly handle the word reordering between words in source and target language, we use the concept of the so-called inverted alignments as given in (Ney et al., 2000).
0.9996860280039077
ref_text =  The details are given in (Och and Ney, 2000).
0.9996379851935089
ref_text =  We use a solution to this problem similar to the one presented in (Och et al., 1999), where target words are joined during training.
0.9996348379269554
ref_text =  For a given partial hypothesis (C; j), the order in which the cities in C have been visited can be ignored (except j), only the score for the best path reaching j has to be stored.
0.9996292855136879
ref_text =  The computing time, the number of search errors, and the multi-reference WER (mWER) are shown as a function of t0.
0.9996174117526234
re

In [74]:
f = [1,2,3,4,5]
print (f[-1])

5


In [75]:
print (f[-2])

4
